In [30]:
import pandas as pd
import numpy as np
import requests

In [31]:
dataPop = requests.get('https://geo.api.gouv.fr/departements/16/communes')
DFPOP = pd.DataFrame(dataPop.json()) [['nom','population']]
DFREI = pd.read_csv('/home/dune/Téléchargements/Hackathon/REI_2022.csv')

DFP2 = pd.read_csv('/home/dune/Téléchargements/Hackathon/Ressources_HackathonDGFiP_novembre2023/GF-2C/160_CHARENTE_ART-P2.csv'
,header = 1, delimiter=';')
DFP1 = pd.read_csv('/home/dune/Téléchargements/Hackathon/Ressources_HackathonDGFiP_novembre2023/GF-2C/160_CHARENTE_ART-P1.csv'
,header = 1, delimiter=';',encoding='latin-1')
DFC1 = pd.read_csv('/home/dune/Téléchargements/Hackathon/Ressources_HackathonDGFiP_novembre2023/GF-2C/160_CHARENTE_ART-C1.csv'
,header = 1, delimiter=';',encoding='latin-1')
DFA1 = pd.read_csv('/home/dune/Téléchargements/Hackathon/Ressources_HackathonDGFiP_novembre2023/GF-2C/160_CHARENTE_ART-A1.csv'
,header = 1, delimiter=',',encoding='latin-1')


DFA1.columns

/tmp/ipykernel_42410/1393039754.py:3: DtypeWarning: Columns (0,4,16,23,29,32,36,51,72,81,86,96,106,109,112,184,200,205,331,335,339,340,354,355,359,360,364,365,366,370,382,386,390,877,882,883,884,885,886,906) have mixed types. Specify dtype option on import or set low_memory=False.
  DFREI = pd.read_csv('/home/dune/Téléchargements/Hackathon/REI_2022.csv')
/tmp/ipykernel_42410/1393039754.py:5: DtypeWarning: Columns (0,1,2,4,6,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,80) have mixed types. Specify dtype option on import or set low_memory=False.
  DFP2 = pd.read_csv('/home/dune/Téléchargements/Hackathon/Ressources_HackathonDGFiP_novembre2023/GF-2C/160_CHARENTE_ART-P2.csv'
/tmp/ipykernel_42410/1393039754.py:7: DtypeWarning: Columns (0,1,2,4,6,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40

Index(['CSDEP', 'CSDIR', 'CTCN', 'CTCNIA', 'NCCO', 'TYCNGR', 'CTCNGR',
       'COARCL', 'CTADR0', 'NIAF', 'NIAFBT', 'CITPCN', 'LTCN', 'NRTP',
       'DAIMRE', 'DAIMMJ', 'LTCU', 'CITPFI', 'CICNFI', 'CIRGFI', 'CIDPFI',
       'TXCNT0', 'TXCNS0', 'TXCNA0', 'TXCH2', 'TXCNU0', 'TXCNZ2', 'TXCH24',
       'TXCH23', 'TXCH0', 'TXCH1', 'MOCHD', 'MNDPDF', 'MNCHDF', 'MNCHD',
       'MBTP01', 'MBTP02', 'MBTP03', 'MBTP04', 'MBTP05', 'MBTP06', 'MBTP07',
       'MBTP08', 'MBTP09', 'MBTP10', 'MBTP11', 'MBTP12', 'MBTP13', 'MBTP14',
       'MBTP15', 'MBTP16', 'MBTP17', 'MBTP18', 'MBTP19', 'MBTP20', 'MBTP21',
       'MBTP22', 'MBTP23', 'MBTP24', 'FILLER', 'CICNGP', 'FILLER.1', 'BX001A'],
      dtype='object')

In [35]:
filtered_columns = [col for col in DFREI.columns if 'CFE' in col]
filtered_columns.append('Libellé commune')
DFREI = DFREI[filtered_columns]

# P2 : Base brute, réduction, montant rce
DFP2f = pd.concat([DFP2[['MBBS0','MOETA2','MOETR2','CTCN','NCCO']].iloc[0:1],DFP2[['MBBS0','MOETA2','MOETR2','CTCN','NCCO']].iloc[8:]])

# P1 : Base après réduction, base exonérée cfe commune, première exonération, deuxième exonération, base taxable commune
DFP1f = pd.concat([DFP1[['MBBSR', 'MNTPXC', 'CNEX01', 'CNEX02','MOBS20','CTCN','NCCO']].iloc[0:1],DFP1[['MBBSR', 'MNTPXC', 'CNEX01', 'CNEX02','MOBS20','CTCN','NCCO']].iloc[8:]])

# C1 : Nature de l'activité
DFC1f = pd.concat([DFC1[['CNAC3','CTCN','NCCO']].iloc[0:1], DFC1[['CNAC3','CTCN','NCCO']].iloc[8:]])

# A1 : Taux CFE commune
DFA1f = DFA1[['TXCNU0','CTCN','LTCN']]

In [36]:
merged_df = pd.merge(DFP2f, DFP1f, on=['NCCO','CTCN'], how='inner')
merged_df = pd.merge(merged_df, DFC1f, on=['NCCO','CTCN'], how='inner')
merged_df = pd.merge(merged_df, DFA1f, on=['CTCN'], how='left')
merged_df = merged_df.dropna()

colonnes = ['MBBS0', 'MOETA2', 'MOETR2', 'CTCN', 'NCCO', 'MBBSR', 'MNTPXC','CNEX01', 'CNEX02', 'MOBS20', 'CNAC3', 'TXCNT0']

merged_df[['CTCN','LTCN','TXCNU0','NCCO','CNAC3','MBBS0','MBBSR', 'MNTPXC', 'MOETA2', 'MOETR2','CNEX01', 'CNEX02', 'MOBS20']]

merged_df

,MBBS0,MOETA2,MOETR2,CTCN,NCCO,MBBSR,MNTPXC,CNEX01,CNEX02,MOBS20,CNAC3,TXCNU0,LTCN
0,base brute cfe annee n (10 car.),montant reduction artisans 10 car.,montant rce 10 car.,CODE COMMUNE DE L'AFT,numero de compte fichier coll CFE,base brute reduite n,base exoneree CFE commune,CODE NATURE PREMIERE EXO,CODE NATURE DEUXIEME EXO,base taxable commune n 10 car.,CODE NATURE ACTIVITE. APE NACE (FRP),taux cfe unique applicable 8 car. ...,Libelle commune au 1/1
5115,0,0,0,101,9930,0,0,,,0,6190Z,2572000,CLAIX
5116,0,0,0,101,9931,0,0,,,0,99904,2572000,CLAIX
5117,1268,0,0,101,9932,1268,0,,,0,4399C,2572000,CLAIX
5118,279,0,0,101,9933,279,0,,,0,3513Z,2572000,CLAIX
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8180,0,0,0,287,21441,0,0,,,0,8299Z,2572000,ROULLET ST ESTEPHE
8181,0,0,0,287,21442,0,0,MIN,,0,7410Z,2572000,ROULLET ST ESTEPHE
8182,0,0,0,287,21443,0,0,,,0,7490B,2572000,ROULLET ST ESTEPHE
8183,0,0,0,287,21444,0,0,,,0,6820A,2572000,ROULLET ST ESTEPHE


In [49]:
DFPOP.sort_values('population')[350:]

,nom,population
51,Brie,4222
251,Roullet-Saint-Estèphe,4295
147,Jarnac,4447
21,Barbezieux-Saint-Hilaire,4751
68,Champniers,5155
146,L'Isle-d'Espagnac,5633
136,Gond-Pontouvre,6017
254,Ruelle-sur-Touvre,7364
304,Saint-Yrieix-sur-Charente,7499
100,La Couronne,7742


In [56]:
nombre_occurrences = len(merged_df[merged_df["CTCN"] == '166'])
print(nombre_occurrences)

426
